In [14]:
#=============================================================================#
#                                      Input                                  #
#=============================================================================#
import importlib
import functions
importlib.reload(functions)

BASE_PATH = "C:/Users/aelek/source/antoniaelek/fantasy-premier-league/"
SEASON = "2018-19"
GWCNT = 5
RANGE_START = 1
RANGE_END = -1

In [15]:
#=============================================================================#
#                                   Get data                                  #
#=============================================================================#
ALLDF = functions.get_cumulative_data(base_path=BASE_PATH, season=SEASON)
PLAYERS = ALLDF["full_name"].values.tolist()
PLAYERS2 = ALLDF["full_name2"].values.tolist()
PLAYERS3 = ALLDF["full_name3"].values.tolist()
ALLDF = ALLDF.set_index(['full_name3'])
PPA = functions.points_per_appearance(BASE_PATH, SEASON)
PPA = PPA[numpy.isnan(PPA.vpc_ratio) == False]
PPA = PPA[PPA.total_points_x >= 0]
PPA_GKP=PPA[PPA.position=='Goalkeeper']
PPA_DEF=PPA[PPA.position=='Defender']
PPA_MID=PPA[PPA.position=='Midfielder']
PPA_FWD=PPA[PPA.position=='Forward']
#PPA = PPA[(PPA.vpc_ratio >= 0.2) | (PPA.total_points_x >= 10)]

In [96]:
# PPA[['name','position']]

In [18]:
#=============================================================================#
#                                   CPV plot                                  #
#=============================================================================#
from bokeh.plotting import figure, output_file, show
from bokeh.layouts import gridplot, widgetbox, layout, row, column
from bokeh.models import CategoricalColorMapper, ColumnDataSource, Range1d, Plot, LinearAxis, Grid
from bokeh.palettes import d3
from bokeh.io import *
import numpy

from bokeh.transform import jitter

def display_vpc(df, out_file):
    palette = d3['Category10'][len(PPA['position'].unique())]
    color_map = CategoricalColorMapper(factors=ALLDF['position'].unique(), palette=palette)

    # set data source for visual
    source = ColumnDataSource(data=dict(
        x=df['now_cost']/10,
        y=df['points_per_game'],
        names=df['name'],
        position=df['position'],
        points_per_game=df['points_per_game'],
        vpc_ratio=df['vpc_ratio'],
        now_cost=df['now_cost']/10,
        circle_size=df['vpc_ratio']/3,#numpy.full((len(df['name']), 1), 0.2),#df['vpc_ratio']/2,
        fill_alpha = df['vpc_ratio']/3))#numpy.full((len(df['name']), 1), 0.6)))

    TOOLS="hover,crosshair,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select,"

    p = figure(tools=TOOLS, x_axis_label = "cost", y_axis_label = "avg points", x_range=(3, 15), y_range=(0, 10))
    p.hover.tooltips = [
        ("Name", "@names"),
        ("Position", "@position"),
        ("Cost", "@now_cost"),
        ("Avg points", "@points_per_game"),
        ("Value Per Cost", "@vpc_ratio")
    ]

    p.scatter(x='x', y='y', radius='circle_size', fill_alpha='fill_alpha', color={'field': 'position', 'transform': color_map}, source=source)

    fig = row(p)
    #output_notebook()
    #show(fig)
    output_file(out_file)
    save(fig)

display_vpc(PPA, BASE_PATH + '/docs/assets/bokeh/value_per_cost.html')
display_vpc(PPA_GKP, BASE_PATH + '/docs/assets/bokeh/value_per_cost_gkp.html')
display_vpc(PPA_DEF, BASE_PATH + '/docs/assets/bokeh/value_per_cost_def.html')
display_vpc(PPA_MID, BASE_PATH + '/docs/assets/bokeh/value_per_cost_mid.html')
display_vpc(PPA_FWD, BASE_PATH + '/docs/assets/bokeh/value_per_cost_fwd.html')

In [16]:
#=============================================================================#
#                                     Stats                                   #
#=============================================================================#
for i in range(0,len(PLAYERS)):
    PLAYER=PLAYERS[i]
    PLAYER2=PLAYERS2[i]
    PLAYER3=PLAYERS3[i]
    
    val = (ALLDF[ALLDF.index.str.startswith(PLAYER3)])
    #print(val.index[0])
    
    md_file=BASE_PATH + 'docs/assets/bokeh/player_stats/' + PLAYER3 + '_text.html'
        
    with open(md_file, 'w', encoding='utf8') as f:
        f.write('<head><style>' + 
                'body, table, td { font-family: "Open Sans", "Helvetica Neue", Helvetica, Arial, sans-serif; color: #444; line-height: 0.8; font-size: 12px; } ' + 
                'h1, h2, h3, h4, h5, h6 { margin: 10px 0 5px 0 !important; font-family: Helvetica, Arial, sans-serif; color: #34465d; font-weight: bold; line-height: 0.8; } ' +
                'h1{font-size:20px} h2{font-size:18px} h3{font-size:16px} h4{font-size:14px} h5{font-size:12px} h6{font-size:12px} ' + 
                'td{ padding: 2px 5px 2px 5px; }</style></head>')
        f.write('<body><h1>' + val['full_name'][0] + '</h1>')                         
        f.write('<table><tr><td colspan=2><h3>' + str(val['position'][0])     + ', ' + str(val['price'][0]) + ' £</h3></td><td></td></tr>')
        f.write('<tr><td><b>Team: </b>'              + str(val['team_name'][0])    + '</td>' +                      
                           '<td><b>Status:</b> ' + str(val['avail_status'][0]) + '</td></tr>')

        f.write('<tr><td colspan=2><h3>Basic Stats</h3></td><td id="basic_stats" style="display:none;">'+ str(val['basic_stats'][0]) +'</td><td></td></tr>')
        f.write('<tr><td><b>Minutes played:</b> '             + str(val['minutes'][0])         + '</td>' +  
                           '<td><b>Total Points:</b> '        + str(val['total_points'][0])    + '</td></tr>' +  
                       '<tr><td><b>Bonus Points:</b> '        + str(val['bonus'][0])           + '</td>' +          
                           '<td><b>BPS:</b> '                 + str(val['bps'][0])             + '</td></tr>' +   
                       '<tr><td><b>Points Per Game:</b> '     + str(val['points_per_game'][0]) + '</td>' +         
                           '<td><b>Times in Dream Team:</b> ' + str(val['dreamteam_count'][0]) + '</td></tr>')      

        f.write('<tr><td colspan=2><h3>Quality</h3></td><td id="quality" style="display:none;">'+ str(val['quality'][0]) +'</td><td></td></tr>')
        f.write('<tr><td><b>Creativity:</b> ' + str(val['creativity'][0])      + '</td>' +       
                           '<td><b>Form:</b> '       + str(val['form'][0])       + '</td></tr>' +       
                       '<tr><td><b>ICT index:</b> '  + str(val['ict_index'][0])  + '</td>' +         
                           '<td><b>Threat:</b> '     + str(val['threat'][0])     + '</td></tr>' +                   
                       '<tr><td><b>Influence:</b> '  + str(val['influence'][0])  + '</td>'+
                           '<td></td></tr>')

        f.write('<tr><td colspan=2><h3>In-Game Stats</h3></td><td id="in_game_stats" style="display:none;">'+ str(val['in_game_stats'][0]) +'</td><td></td></tr>')
        f.write('<tr><td><b>Goals Scored:</b> '                + str(val['goals_scored'][0])          + '</td>' +  
                            '<td><b>Assists:</b> '             + str(val['assists'][0])          + '</td></tr>' +     
                        '<tr><td><b>Goals Conceded:</b> '      + str(val['goals_conceded'][0])   + '</td>' + 
                            '<td><b>Clean Sheets:</b> '        + str(val['clean_sheets'][0])     + '</td></tr>' +         
                        '<tr><td><b>Saves:</b> '               + str(val['saves'][0])            + '</td>' +
                            '<td><b>Own Goals:</b> '           + str(val['own_goals'][0])        + '</td></tr>' + 
                        '<tr><td><b>Penalties Missed:</b> '    + str(val['penalties_missed'][0]) + '</td>' +    
                            '<td><b>Penalties Saved:</b> '     + str(val['penalties_saved'][0])  + '</td></tr>' +
                        '<tr><td><b>Yellow Cards:</b> '        + str(val['yellow_cards'][0])     + '</td>' +
                            '<td><b>Red Cards:</b> '           + str(val['red_cards'][0])        + '</td></tr>')

        f.write('<tr><td colspan=2><h3>Popularity</td><td id="popularity" style="display:none;">'+ str(val['popularity'][0]) +'</td><td></td></tr>')
        f.write('<tr><td><b>Transfers In:</b> '        + str(val['transfers_in'][0])       + '</td>' +                  
                    '<td><b>Transfers Out:</b> '       + str(val['transfers_out'][0])       + '</td></tr>' +                           
                '<tr><td><b>GW Transfers In:</b> '     + str(val['transfers_in_event'][0])  + '</td>' +        
                    '<td><b>GW Transfers Out:</b> '    + str(val['transfers_out_event'][0]) + '</td></tr>' +    
                '<tr><td><b>Selected by:</b> '       + str(val['selected_by_percent'][0]) + '% </td>' +
                    '<td></td></tr></table></body>') 

In [17]:
#=============================================================================#
#                                     Plots                                   #
#=============================================================================#
from bokeh.plotting import figure, output_file, show
from bokeh.layouts import gridplot, widgetbox, layout, row, column
from bokeh.models import CustomJS, ColumnDataSource, Range1d, Plot, LinearAxis, Grid
from bokeh.models.glyphs import ImageURL
from bokeh.models.widgets import Button, RadioButtonGroup, Select, RangeSlider
from bokeh.io import *
from IPython.display import display, Markdown
from IPython.display import HTML, display
from matplotlib import pyplot as plt

for i in range(0,len(PLAYERS)):
    PLAYER=PLAYERS[i]
    PLAYER2=PLAYERS2[i]
    PLAYER3=PLAYERS3[i]
    
#     if PLAYER != 'David De Gea':
#        continue
    
    # get plots data
    val = functions.get_player_data(base_path=BASE_PATH, player=PLAYER2)
    source = ColumnDataSource(data=dict(x=val['gw'], y_value=val['value']/10, y_pts=val['total_points'], y_mins=val['minutes'], y_influence=val['influence']))
    
    # plots
    p_price = figure(y_axis_label='£', tools="", x_range=(1, GWCNT))
    p_price.line('x', 'y_value', source=source, line_width=2)

    p_pts = figure(y_axis_label='pts', tools="", x_range=(1, GWCNT))
    p_pts.line('x', 'y_pts', source=source, line_width=2)

    p_mins = figure(y_axis_label='mins', tools="", x_range=(1, GWCNT), y_range=(0, 100))
    p_mins.line('x', 'y_mins', source=source, line_width=2)

    p_ict = figure(x_axis_label='gw', y_axis_label='influence', tools="", x_range=(1, GWCNT))
    p_ict.line('x', 'y_influence', source=source, line_width=2)

    p = gridplot([p_price, p_pts, p_mins, p_ict],ncols=1, plot_width=300, plot_height=110, toolbox=None, toolbar_options={'logo': None})

    fig = row(p)
#     output_notebook()
#     show(fig)
    output_file(BASE_PATH + '/docs/assets/bokeh/player_stats/' + PLAYER3 + '.html')
#     print(PLAYER3)
    save(fig)